In [1]:
import os
import pandas as pd
import numpy as np
import math
from datetime import datetime
import datetime as dt
import tensorflow as tf
from tensorflow import keras
from keras.layers import GRU, Dropout, SimpleRNN, LSTM, Dense, SimpleRNN, GRU
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn

import plotly as py
import plotly.io as pio

# Mute sklearn warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)

from plotly.offline import init_notebook_mode

# Show charts when running kernel
init_notebook_mode(connected=True)

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import seaborn as sns
import matplotlib.pyplot as plt
from colorama import Fore

/Users/rayhanzirvi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [19]:
# Path to your CSV file
csv_file_path = '/Users/rayhanzirvi/Desktop/coal_consumption_energy_source.csv'
# csv_file_path = 'math-ysics/SpringData2024/data/consume-produce_data/annual_generation_state.csv'

# Read the CSV file into a DataFrame
df_coal = pd.read_csv(csv_file_path)

# for col in df.columns:
#     print(col)

# Display the first few rows of the DataFrame
# print(df.head())

coal_2021 = df_coal[['State', '2021']]

print(coal_2021)

   State        2021
0     AK      18,694
1     AL     309,791
2     AR     216,123
3     AZ     160,299
4     CA      28,244
5     CO     252,442
6     CT       2,880
7     DC           0
8     DE       4,542
9     FL     200,193
10    GA     203,870
11    HI      12,566
12    IA     264,419
13    ID       3,051
14    IL     521,615
15    IN     753,557
16    KS     219,031
17    KY     548,443
18    LA      95,856
19    MA           0
20    MD      69,186
21    ME       1,588
22    MI     436,203
23    MN     179,055
24    MO     616,413
25    MS      64,446
26    MT     122,765
27    NC     222,501
28    ND     361,811
29    NE     216,298
30    NH       3,259
31    NJ      12,586
32    NM     133,228
33    NV      35,910
34    NY       5,370
35    OH     575,920
36    OK     131,695
37    OR       1,303
38    PA     485,193
39    RI           0
40    SC     162,628
41    SD      21,589
42    TN     225,784
43    TX     968,401
44    UT     276,159
45    VA      68,603
46    VT     

In [20]:
# Path to your CSV file
csv_file_path = '/Users/rayhanzirvi/Desktop/petroleum_consumption_energy_source.csv'
# csv_file_path = 'math-ysics/SpringData2024/data/consume-produce_data/annual_generation_state.csv'

# Read the CSV file into a DataFrame
df_petroleum = pd.read_csv(csv_file_path)

# for col in df.columns:
#     print(col)

# Display the first few rows of the DataFrame
# print(df.head())

petroleum_2021 = df_petroleum[['State', '2021']]

print(petroleum_2021)

   State        2021
0     AK     256,041
1     AL     580,020
2     AR     329,239
3     AZ     606,053
4     CA   2,969,515
5     CO     489,206
6     CT     287,268
7     DC      17,525
8     DE     113,210
9     FL   1,750,506
10    GA     922,614
11    HI     222,977
12    IA     411,463
13    ID     188,712
14    IL   1,140,285
15    IN     714,790
16    KS     338,260
17    KY     585,943
18    LA   1,840,317
19    MA     502,317
20    MD     433,109
21    ME     164,149
22    MI     816,074
23    MN     567,228
24    MO     609,448
25    MS     385,435
26    MT     176,856
27    NC     884,691
28    ND     169,566
29    NE     237,542
30    NH     141,670
31    NJ     750,887
32    NM     262,264
33    NV     284,865
34    NY   1,209,837
35    OH   1,029,340
36    OK     519,226
37    OR     322,320
38    PA   1,035,606
39    RI      76,590
40    SC     509,517
41    SD     119,919
42    TN     715,602
43    TX   6,802,215
44    UT     302,537
45    VA     789,358
46    VT     

In [21]:
# Path to your CSV file
csv_file_path = '/Users/rayhanzirvi/Desktop/nuclear_consumption_energy_source.csv'
# csv_file_path = 'math-ysics/SpringData2024/data/consume-produce_data/annual_generation_state.csv'

# Read the CSV file into a DataFrame
df_nuclear = pd.read_csv(csv_file_path)

# for col in df.columns:
#     print(col)

# Display the first few rows of the DataFrame
# print(df.head())

nuclear_2021 = df_nuclear[['State', '2021']]

print(nuclear_2021)

   State       2021
0     AK          0
1     AL    480,897
2     AR    141,603
3     AZ    330,406
4     CA    172,123
5     CO          0
6     CT    179,844
7     DC          0
8     DE          0
9     FL    293,097
10    GA    354,662
11    HI          0
12    IA          0
13    ID          0
14    IL  1,013,204
15    IN          0
16    KS     89,572
17    KY          0
18    LA    180,180
19    MA          0
20    MD    156,623
21    ME          0
22    MI    358,698
23    MN    147,526
24    MO     44,839
25    MS    122,971
26    MT          0
27    NC    450,408
28    ND          0
29    NE     71,875
30    NH    102,957
31    NJ    293,972
32    NM          0
33    NV          0
34    NY    325,671
35    OH    182,628
36    OK          0
37    OR          0
38    PA    792,878
39    RI          0
40    SC    561,696
41    SD          0
42    TN    369,061
43    TX    420,047
44    UT          0
45    VA    298,458
46    VT          0
47    WA     88,909
48    WI    104,149


In [22]:
# Path to your CSV file
csv_file_path = '/Users/rayhanzirvi/Desktop/gas_consumption_energy_source.csv'
# csv_file_path = 'math-ysics/SpringData2024/data/consume-produce_data/annual_generation_state.csv'

# Read the CSV file into a DataFrame
df_gas = pd.read_csv(csv_file_path)

# for col in df.columns:
#     print(col)

# Display the first few rows of the DataFrame
# print(df.head())

gas_2021 = df_gas[['State', '2021']]

print(gas_2021)

   State        2021
0     AK     384,560
1     AL     739,546
2     AR     360,504
3     AZ     484,962
4     CA   2,172,757
5     CO     526,513
6     CT     306,843
7     DC      28,336
8     DE      87,778
9     FL   1,591,156
10    GA     773,889
11    HI         133
12    IA     383,424
13    ID     135,178
14    IL   1,082,481
15    IN     870,075
16    KS     292,156
17    KY     365,718
18    LA   1,861,540
19    MA     404,324
20    MD     302,243
21    ME      57,233
22    MI     950,120
23    MN     523,812
24    MO     293,874
25    MS     576,923
26    MT      86,998
27    NC     637,553
28    ND     190,784
29    NE     191,008
30    NH      60,116
31    NJ     697,019
32    NM     284,989
33    NV     305,212
34    NY   1,361,190
35    OH   1,293,129
36    OK     745,038
37    OR     305,665
38    PA   1,867,361
39    RI     105,473
40    SC     350,088
41    SD      96,787
42    TN     413,619
43    TX   4,736,878
44    UT     274,376
45    VA     699,927
46    VT     

In [23]:
# Path to your CSV file
csv_file_path = '/Users/rayhanzirvi/Desktop/renewable_consumption_energy_source.csv'
# csv_file_path = 'math-ysics/SpringData2024/data/consume-produce_data/annual_generation_state.csv'

# Read the CSV file into a DataFrame
df_renewable = pd.read_csv(csv_file_path)

# for col in df.columns:
#     print(col)

# Display the first few rows of the DataFrame
# print(df.head())

renewable_2021 = df_renewable[['State', '2021']]

print(renewable_2021)

   State        2021
0     AK      24,907
1     AL     303,213
2     AR     114,123
3     AZ     197,117
4     CA   1,334,148
5     CO     216,066
6     CT      51,792
7     DC       3,358
8     DE       8,136
9     FL     353,523
10    GA     332,568
11    HI      35,105
12    IA     599,196
13    ID     145,902
14    IL     335,451
15    IN     212,543
16    KS     276,717
17    KY     102,456
18    LA     144,874
19    MA     103,215
20    MD      73,820
21    ME     133,128
22    MI     247,406
23    MN     297,813
24    MO     144,808
25    MS      68,941
26    MT     134,741
27    NC     288,212
28    ND     185,516
29    NE     217,397
30    NH      50,415
31    NJ      85,852
32    NM     138,630
33    NV     139,948
34    NY     473,907
35    OH     165,301
36    OK     367,146
37    OR     449,244
38    PA     223,410
39    RI      14,546
40    SC     170,674
41    SD     205,338
42    TN     198,765
43    TX   1,293,066
44    UT      67,954
45    VA     189,262
46    VT     

In [37]:
# Assume that the dataframes are named df_petroleum, df_gas, df_nuclear, df_coal, and df_renewable
# And each dataframe has a '2021' column with the energy values and a 'State' column for the state identifier

# First, create a list of your dataframes and corresponding suffixes to avoid the overlapping suffix issue
dataframes = {
    'petroleum_2021': '_petroleum',
    'gas_2021': '_gas',
    'nuclear_2021': '_nuclear',
    'coal_2021': '_coal',
    'renewable_2021': '_renewable',
}

# Now, iteratively merge your dataframes on 'State'
df_final = None
for name, suffix in dataframes.items():
    if df_final is None:
        df_final = globals()[name]
    else:
        df_final = df_final.merge(globals()[name], on='State', suffixes=('', suffix))

# Calculate the total energy for each state by summing all the '2021' columns except '2021_renewable'
energy_columns = [col for col in df_final.columns if '2021' in col and '_renewable' not in col]
df_final['total_energy'] = df_final[energy_columns].sum(axis=1)

# Calculate the percentage of renewable energy
# Assuming the column for renewable energy in the merged dataframe is '2021_renewable'
df_final['percentage_renewable'] = (df_final['2021_renewable'] / df_final['total_energy']) * 100

# print(df_final['total_energy'])
print(df_final[['State', 'total_energy', 'percentage_renewable']])

   State  total_energy  percentage_renewable
0     AK        659295              3.777823
1     AL       2110254             14.368555
2     AR       1047469             10.895120
3     AZ       1581720             12.462193
4     CA       5342639             24.971704
5     CO       1268161             17.037742
6     CT        776835              6.667053
7     DC         45861              7.322126
8     DE        205530              3.958546
9     FL       3834952              9.218447
10    GA       2255035             14.747798
11    HI        235676             14.895450
12    IA       1059306             56.564959
13    ID        326941             44.626400
14    IL       3757585              8.927303
15    IN       2338422              9.089164
16    KS        939019             29.468733
17    KY       1500104              6.829926
18    LA       3977893              3.641978
19    MA        906641             11.384330
20    MD        961161              7.680295
21    ME  